В этом ноубуке используются обченные тестовые модели, с их помощью происходит сэмплирование и далее закодированные категориальные переменные переводятся в изначальные. После чего замеряется качество.

In [1]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ohe_noise.tabddpm_ohe_noise import *

from tqdm.notebook import tqdm

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

### Подготовка

In [2]:
dataname = 'adult_ON'
model_short = 'tabddpm_ON'

In [3]:
if torch.cuda.is_available():
    device = f'cuda'
else:
    device = 'cpu'
CONFIG.add_arg('device', device)
print(f"Device: {device}")

CONFIG.add_arg('dataname', dataname)
CONFIG.add_arg('real_path',
                   f'./synthetic/{dataname}/initial_real.csv')
CONFIG.add_arg('test_path',
                   f'./synthetic/{dataname}/initial_test.csv')

CONFIG.add_arg('info_path',
                   f'./data/{dataname}/initial_info.json')

Device: cuda


In [4]:
model_save_path_hist = []
for sigma in [0, 0.001, 0.01, 0.1, 0.25, 0.5]:
    model_save_path=f"./models/{model_short}/ckpt/{model_short}_const_{str(sigma).replace('.', '_')}"
    model_save_path_hist.append(model_save_path)

for i, sigmas in enumerate(range(8)):
    model_save_path=f"./models/{model_short}/ckpt/{model_short}_mult_{i}"
    model_save_path_hist.append(model_save_path)

### Sample + Eval

In [12]:
CONFIG.get_all_args()

{'dataname': 'adult_ON',
 'method': 'tabddpm_ON_mult_6',
 'device': 'cuda',
 'save_path': './synthetic/adult_ON/initial_tabddpm_ON_mult_6.csv',
 'mode': 'train',
 'train': 1,
 'sample_save_path': './synthetic/adult_ON/initial_tabddpm_ON_mult_6.csv',
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 103,
 'real_path': './synthetic/adult_ON/initial_real.csv',
 'test_path': './synthetic/adult_ON/initial_test.csv',
 'info_path': './data/adult_ON/initial_info.json'}

In [1]:
overall_metrics = {}

for model_save_path in model_save_path_hist:
    model_name = model_save_path.split('/')[-1]
    sample_save_path = f'./synthetic/{dataname}/{model_name}.csv'

    CONFIG.add_arg('method', model_name)
    CONFIG.add_arg('dataname', dataname)
    CONFIG.add_arg('save_path',
                       f"synthetic/{CONFIG.get_arg('dataname')}/{CONFIG.get_arg('method')}.csv")
    CONFIG.add_arg('sample_save_path',
                       f"synthetic/{CONFIG.get_arg('dataname')}/{CONFIG.get_arg('method')}.csv")

    overall_metrics[model_name] = {}

    print(model_save_path, model_name, sample_save_path)

    tabddpm_noise_ohe = TabDDPM_OHE_Noise(CONFIG, model_save_path=model_save_path, sigmas=None)
    tabddpm_noise_ohe.sample(sample_save_path=sample_save_path)
    postsample_OHE(dataname, f'./synthetic/{dataname}/initial_{model_name}.csv')

    # подсчет метрик
    # overall_metrics[model_name]['base_metrics'] = calculate_base_metrics(make_binary=True, value=' >50K') - ошибка
    overall_metrics[model_name]['similarity'] = calculate_similarity()
    overall_metrics[model_name]['mle'] = calculate_mle()
    overall_metrics[model_name]['detection'] = calculate_detection()
    overall_metrics[model_name]['DCR'] = calculate_DCR()
    overall_metrics[model_name]['quality'] = calculate_alpha_beta()

In [9]:
final_metrics_table = []
for m in overall_metrics.keys():
    # Сбор таблички результатов
    tmp = pd.DataFrame([{'Model':'TabDDPM ON', 'Type':m, 'Data':dataname}])
    tmp.columns = pd.MultiIndex.from_tuples([('', i) for i in tmp.columns])
    result = [tmp]
    
    for metric_group in overall_metrics[m].keys():
        tmp = pd.DataFrame([overall_metrics[m][metric_group]])
        tmp.columns = pd.MultiIndex.from_tuples([(metric_group, i) for i in tmp.columns])
        result.append(tmp)
    result = pd.concat(result, axis = 1)
    final_metrics_table.append(result)
    
final_metrics_table = pd.concat(final_metrics_table)

In [10]:
final_metrics_table.round(3)

similarity  \
        Model                    Type      Data Column Shapes Score, %   
0  TabDDPM ON      tabddpm_ON_const_0  adult_ON                 37.138   
0  TabDDPM ON  tabddpm_ON_const_0_001  adult_ON                 39.300   
0  TabDDPM ON   tabddpm_ON_const_0_01  adult_ON                 41.973   
0  TabDDPM ON    tabddpm_ON_const_0_1  adult_ON                 26.846   
0  TabDDPM ON   tabddpm_ON_const_0_25  adult_ON                 41.642   
0  TabDDPM ON    tabddpm_ON_const_0_5  adult_ON                 30.504   
0  TabDDPM ON       tabddpm_ON_mult_0  adult_ON                 31.363   
0  TabDDPM ON       tabddpm_ON_mult_1  adult_ON                 42.049   
0  TabDDPM ON       tabddpm_ON_mult_2  adult_ON                 34.944   
0  TabDDPM ON       tabddpm_ON_mult_3  adult_ON                 60.711   
0  TabDDPM ON       tabddpm_ON_mult_4  adult_ON                 38.147   
0  TabDDPM ON       tabddpm_ON_mult_5  adult_ON                 36.191   
0  TabDDPM ON       tabddpm_ON_mult_6  adult_ON                 28.265   

                                                          \
  Column Pair Trends Score, % Overall Score (Average), %   
0                      24.722                     30.930   
0                      25.886                     32.593   
0                      26.270                     34.121   
0                      21.053                     23.950   
0                      26.223                     33.933   
0                      23.332                     26.918   
0                      22.219                     26.791   
0                      26.293                     34.171   
0                      23.343                     29.143   
0                      39.068                     49.890   
0                      26.036                     32.091   
0                      24.591                     30.391   
0                      20.255                     24.260   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                              62.862   
0                                              60.700   
0                                              58.027   
0                                              73.154   
0                                              58.358   
0                                              69.496   
0                                              68.637   
0                                              57.951   
0                                              65.056   
0                                              39.289   
0                                              61.853   
0                                              63.809   
0                                              71.735   

                                                           \
  Error rate (%) of column-wise density estimation std, %   
0                                                  24.124   
0                                                  22.229   
0                                                  21.869   
0                                                  21.806   
0                                                  21.725   
0                                                  25.764   
0                                                  26.868   
0                                                  21.911   
0                                                  23.308   
0                                                  34.564   
0                                                  24.116   
0                                                  26.570   
0                                                  22.543   

                                                           \
  Error rate (%) of pair-wise column correlation score, %   
0                                                  75.278   
0                                                  74.114   
0                               

In [17]:
final_metrics_table.to_csv(f'./eval/total/{dataname}/{model_short}_final_metrcs_table.csv', index=False)

---
#### Пример ошибки

In [12]:
calculate_base_metrics(make_binary=True, value=' >50K')

/venv/main/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ValueError: Found unknown categories [' Doctorate', ' Prof-school', ' Assoc-acdm', ' Some-college', ' Preschool', ' Assoc-voc', ' Bachelors', ' HS-grad', ' Masters'] in column 0 during transform